**Falta implementar de manera correcta GPT-OSS**

In [1]:
import torch
dev = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(dev)

cuda


In [5]:
print(torch.cuda.get_device_name())
print(torch.cuda.get_device_capability())

NVIDIA RTX 5000 Ada Generation
(8, 9)


In [7]:
%%time
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("openai/gpt-oss-20b")
model = AutoModelForCausalLM.from_pretrained("openai/gpt-oss-20b")
messages = [
    {"role": "user", "content": "Who are you?"},
]
inputs = tokenizer.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(model.device)

outputs = model.generate(**inputs, max_new_tokens=40)
print(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:]))

/home/flopezp/.venvs/foo/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CPU times: user 1.01 s, sys: 195 ms, total: 1.21 s
Wall time: 1min 22s


ImportError: Using mxfp4 requires Accelerate: `pip install accelerate`